In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#@title cd
%pwd
%cd /content/drive/MyDrive/afeka/Project/code/PTEEnet/PTEEnet

/content/drive/MyDrive/afeka/Project/code/PTEEnet/PTEEnet


In [3]:
#@title install packages
#%pip install chainer
%pip install -Uqq ipdb

     |████████████████████████████████| 792 kB 9.9 MB/s 
     |████████████████████████████████| 375 kB 63.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.26 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.31.1 which is incompatible.


In [4]:
#@title read gpu details
#import chainer
#chainer.print_runtime_info()

#print('GPU availability:', chainer.cuda.available)
#print('cuDNN availablility:', chainer.cuda.cudnn_enabled)
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Sat Jan 29 18:51:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
#@title imports
"""
high level support for doing this and that.
"""
from __future__ import print_function
import time
import csv
import numpy as np
import torch
import torch.nn.functional as F
from scipy import stats
from init import initializer
from eenet import EENet
from custom_eenet import CustomEENet
from gatednet import GatedNet
import matplotlib.pylab as plt
import loss_functions
import utils
import config
import gc
import sys
import math
import os
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms.functional as tf
import matplotlib.pyplot as plt
import ipdb

#import xgboost as xgb
#from sklearn.metrics import confusion_matrix, classification_report
#from sklearn.metrics import accuracy_score
#import joblib as jobl
from collections import Counter
from matplotlib import pyplot
from numpy import where
#from imblearn.over_sampling import RandomOverSampler 
#from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
#from xgboost.sklearn import XGBRegressor

%matplotlib inline


In [6]:
#@title train_gated_xgboost
"""
params={
'booster':'gbtree',
'objective': 'multi:softmax',
'num_class':11,
'gamma':0.6,#0.1,
'max_depth':10, #6
'lambda': 3,#2,
'alpha': 4, #new
'subsample':0.7,
'colsample_bytree':0.7,
'min_child_weight':1,
'silent':0 ,
'eta': 0.01, #0.01
'seed':1000,
'tree_method': 'gpu_hist',
'gpu_id': 0,
}
"""
depth = 8
params={
'booster':'gbtree',
'objective': 'multi:softmax',
'num_class':11,
'gamma':0.6,#0.1,
'max_depth':depth, #6
'lambda': 3,#2,
'alpha': 4, #new
'subsample':0.7,
'colsample_bytree':0.7,
'min_child_weight':1,
'silent':0 ,
'eta': 0.03, #0.01
'seed':1000,
'tree_method': 'gpu_hist',
'gpu_id': 0,
}

#82%

def evaluate(model, X, Y):#, weights):
    predicted_Y = model.predict(X)
    #ipdb.set_trace(context=6)
    print(predicted_Y)
    accuracy = accuracy_score(Y, predicted_Y)#, weights)
    return accuracy
"""
def produce_weights(data, size, num_of_classes):
    counter = Counter(data)
    weights = np.zeros(num_of_classes)
    weights_vec = np.zeros(size)
    for key in counter:
        #print(key)
        weights[int(key)] = counter[key]
    weights = 1 - weights / np.max(weights) + 0.001
    #ipdb.set_trace(context=6)
    #print(weights)
    for i, value in enumerate(data):
        weights_vec[i] = weights[value]
    return weights_vec
""" 

def train_gated_xgboost(args, noise_levels):
    log_file = open(args.gated_models_dir+'/train_gated_xgboost_log.txt', 'a', newline='')
    log_file.write("Training gated xgboost mode:\n")
    log_file.write("loss threshold: " + str(args.loss_threshold) + ", SNR=" + str(args.noise_snr) + "\n")
    
    %matplotlib inline

    type_loader = 'test'
    ground_truth_noisy_dir = args.ground_truths_dir + '/' + str(args.loss_threshold)# + '/' + 'snr_' + str(noise_levels[0])
    """
    data_file_train = ground_truth_noisy_dir + '/datas_train.' + str(args.loss_threshold) + '.pt'
    data_train = torch.load(data_file_train)
    print('loaded data train at: ', data_file_train)
    
    gt_file_train = ground_truth_noisy_dir + '/exit_idx_predictions_train.' + str(args.loss_threshold) + '.pt'
    exits_ground_truth_train = torch.load(gt_file_train)
    print('loaded exits ground truths train at: ', gt_file_train)
    """
    data_file_test = ground_truth_noisy_dir + '/datas_' + type_loader + '.' + str(args.loss_threshold) + '.pt'
    data_test = torch.load(data_file_test)
    print('loaded data test at: ', data_file_test)
    
    gt_file_test = ground_truth_noisy_dir + '/exit_idx_predictions_' + type_loader + '.' + str(args.loss_threshold) + '.pt'
    exits_ground_truth_test = torch.load(gt_file_test)
    print('loaded exits ground truths test at: ', gt_file_test)
    
    if 0:
        for m in range(args.num_noise_levels-1):
            ground_truth_noisy_dir = args.ground_truths_dir + '/' + str(args.loss_threshold) + '/' + 'snr_' + str(noise_levels[m+1])
            """
            data_file_train = ground_truth_noisy_dir + '/datas_train.' + str(args.loss_threshold) + '.pt'
            data = torch.load(data_file_train)
            print('loaded data train at: ', data_file_train)
            data_train = np.concatenate((data_train, data), axis=0)

            gt_file_train = ground_truth_noisy_dir + '/exit_idx_predictions_train.' + str(args.loss_threshold) + '.pt'
            data = torch.load(gt_file_train)
            print('loaded exits ground truths train at: ', gt_file_train)
            exits_ground_truth_train = np.concatenate((exits_ground_truth_train, data), axis=0)
            """
            data_file_test = ground_truth_noisy_dir + '/datas_' + type_loader + '.' + str(args.loss_threshold) + '.pt'
            data = torch.load(data_file_test)
            print('loaded data test at: ', data_file_test)
            data_test = np.concatenate((data_test, data), axis=0)

            gt_file_test = ground_truth_noisy_dir + '/exit_idx_predictions_' + type_loader + '.' + str(args.loss_threshold) + '.pt'
            data = torch.load(gt_file_test)
            print('loaded exits ground truths test at: ', gt_file_test)
            exits_ground_truth_test = np.concatenate((exits_ground_truth_test, data), axis=0)
        
    """
    ipdb.set_trace(context=6)
    gt_file_train = args.ground_truths_dir + '/exit_idx_predictions_train.' + str(args.loss_threshold) + '.pt'
    exits_ground_truth_train = torch.load(gt_file_train)
    print('loaded exits ground truths at: ', gt_file_train)
    train_len = len(exits_ground_truth_train)
    #print('exits_ground_truth_train {:8d}'.format(train_len))
    
    gt_file_test = args.ground_truths_dir + '/exit_idx_predictions_test.' + str(args.loss_threshold) + '.pt'
    exits_ground_truth_test = torch.load(gt_file_test)
    print('loaded exits ground truths at: ', gt_file_test)
    test_len = len(exits_ground_truth_test)
    #print('exits_ground_truth_test {:8d}'.format(test_len))
    
    data_file_train = args.ground_truths_dir + '/datas_train.' + str(args.loss_threshold) + '.pt'
    data_train = torch.load(data_file_train)
    print('loaded data train at: ', data_file_train)
    data_train_len = len(data_train)
    #print('data_train_len {:8d}'.format(data_train_len))
    
    data_file_test = args.ground_truths_dir + '/datas_test.' + str(args.loss_threshold) + '.pt'
    data_test = torch.load(data_file_test)
    print('loaded data test at: ', data_file_test)
    data_test_len = len(data_test)
    #print('data_test_len {:8d}'.format(data_test_len))
    """
    #ipdb.set_trace(context=6)
    #data_train = torch.stack(data_train, dim=0)
    #data_test = torch.stack(data_test, dim=0)
    #d1, d2, d3, d4 = data_train.shape
    #data_train = data_train.reshape((d1, d2*d3*d4))
    d1, d2, d3, d4 = data_test.shape
    data_test = data_test.reshape((d1, d2*d3*d4))
    #data_train = torch.flatten(torch.from_numpy(data_train), start_dim=1)
    #data_test = torch.flatten(torch.from_numpy(data_test), start_dim=1)

    #data_train = data_train.cpu().detach().numpy()
    #data_test = data_test.cpu().detach().numpy()
    #exits_ground_truth_train = np.array(exits_ground_truth_train)
    exits_ground_truth_test = np.array(exits_ground_truth_test)
    #ipdb.set_trace(context=6)
    ros_over = RandomOverSampler(random_state=42)
    ros_under = RandomUnderSampler(random_state=42)
    sm = SMOTE(random_state=42)
    
    #data_train, exits_ground_truth_train = ros_over.fit_resample(data_train, exits_ground_truth_train)
    #data_test, exits_ground_truth_test = ros_over.fit_resample(data_test, exits_ground_truth_test)
    #data_train, exits_ground_truth_train = sm.fit_resample(data_train, exits_ground_truth_train)
    print('Resampled test dataset shape before %s' % Counter(exits_ground_truth_test))

    data_test, exits_ground_truth_test = sm.fit_resample(data_test, exits_ground_truth_test)
    #print('Resampled train dataset shape %s' % Counter(exits_ground_truth_train))
    print('Resampled test dataset shape %s' % Counter(exits_ground_truth_test))

    X_train, X_test, y_train, y_test = train_test_split(data_test, exits_ground_truth_test, test_size=0.33, random_state=42)
    data_test = None
    exits_ground_truth_test = None
    
    #ipdb.set_trace(context=6)
    
    #xgb_train = xgb.DMatrix(X_train, label=y_train)#, weight=weights_vec_train)
    #xgb_val = xgb.DMatrix(X_test, label=y_test)#, weight=weights_vec_test)
    #watchlist = [(xgb_train, 'train'), (xgb_val, 'val')]
    
    num_rounds = 1500
    #depth = 4
    # Training phase
    print("building model...")
    XGB = XGBRegressor(tree_method = "gpu_hist", objective = "multi:softmax", num_class=11,\
                       single_precision_histogram=True, max_depth=args.max_depth, eval_metric=["merror"],\
                       n_estimators=num_rounds, verbosity=1)
    #print(XGB)
    eva_set = [(X_test, y_test)]
    X_test = None
    y_test = None
    #ipdb.set_trace(context=6)
    
    #XGB = XGB.fit(X_train, y_train, eval_metric="auc", early_stopping_rounds=100, verbose=True)
    XGB = XGB.fit(X_train, y_train, verbose=True, early_stopping_rounds=5, eval_set=eva_set)
    #score = XGB.score(X_train, y_train)  
    #print("Training score: ", score)
    #plst = list(params.items())
    #gbm = xgb.train(plst, xgb_train, num_rounds, watchlist,early_stopping_rounds=100)
    print("saving model...")
    log_file.write("saving model...\n")
    XGB.save_model(args.gated_models_dir + '/xg_model.' + str(args.loss_threshold) + '.depth' + str(args.max_depth) + '.json')
    #jobl.dump(gbm, args.gated_models_dir + '/xg_model.' + str(args.loss_threshold) + '.bin')
    #validation_accuracy = evaluate(gbm, xgb_val, y_test)#, weights_vec_test)
    #validation_accuracy = evaluate(XGB, X_test, y_test)#, weights_vec_test)
    
    #print("  - validation accuracy = {0:.4f}".format(validation_accuracy))
    #log_file.write("validation accuracy: " + str(validation_accuracy) + "\n")
    log_file.close()
    return
    #return validation_accuracy

In [7]:
#@title load_generated_data
from torch.utils.data import Dataset, TensorDataset, DataLoader

class CustomTensorDataset(Dataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors, transform=None):
        #ipdb.set_trace(context=6)
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.transform = transform

    def __getitem__(self, index):
        x = self.tensors[0][index]

        if self.transform:
            x = self.transform(x)

        y = self.tensors[1][index]
        
        return x, y

    def __len__(self):
        return self.tensors[0].size(0)

def smote_data(data, target, return_to_org_shape=False):
    sm = SMOTE(random_state=42)
    
    d1, d2, d3, d4 = data.shape
    data = data.reshape((d1, d2*d3*d4))
    print('Resampled dataset shape before %s' % Counter(target))
    data, target = sm.fit_resample(data, target)
    print('Resampled dataset shape %s' % Counter(target))

    #ipdb.set_trace(context=6) 
    if return_to_org_shape:
        d1, d5 = data.shape
        data = data.reshape((d1, d2, d3, d4))   

    return data, target
    
        
def load_generated_data(args, smote, return_to_org_shape=False):
    ground_truth_noisy_dir = args.ground_truths_dir + '/' + str(args.loss_threshold)# + '/' + 'snr_' + str(noise_levels[0])
    
    data_file_train = ground_truth_noisy_dir + '/datas_train.' + str(args.loss_threshold) + '.pt'
    data_train = torch.load(data_file_train)
    print('loaded data train at: ', data_file_train)
    
    data_file_test = ground_truth_noisy_dir + '/datas_test.' + str(args.loss_threshold) + '.pt'
    data_test = torch.load(data_file_test)
    print('loaded data test at: ', data_file_test)
    
    gt_file_train = ground_truth_noisy_dir + '/exit_idx_predictions_train.' + str(args.loss_threshold) + '.pt'
    exits_ground_truth_train = torch.load(gt_file_train)
    print('loaded exits ground truths train at: ', gt_file_train)
    
    gt_file_test = ground_truth_noisy_dir + '/exit_idx_predictions_test.' + str(args.loss_threshold) + '.pt'
    exits_ground_truth_test = torch.load(gt_file_test)
    print('loaded exits ground truths test at: ', gt_file_test)

    if smote:
        data_train, exits_ground_truth_train = smote_data(data_train, exits_ground_truth_train, return_to_org_shape)
    else:
        d1, d2, d3, d4 = data_train.shape
        data_train = data_train.reshape((d1, d2*d3*d4))        
    
    data_train, data_test, exits_ground_truth_train, exits_ground_truth_test = train_test_split(data_train,\
                                                                                                exits_ground_truth_train, test_size=0.2, random_state=42)
    
    data_train, target_train = torch.from_numpy(data_train), torch.from_numpy(exits_ground_truth_train)    
    data_test, target_test = torch.from_numpy(data_test), torch.from_numpy(exits_ground_truth_test)
    
    train_set = CustomTensorDataset(tensors=(data_train, target_train))    
    validation_set = CustomTensorDataset(tensors=(data_test, target_test))
    loader_train = torch.utils.data.DataLoader(train_set, batch_size=args.batch_size, shuffle=args.shuffle_train)
    loader_val = torch.utils.data.DataLoader(validation_set, batch_size=args.test_batch, shuffle=args.shuffle_test)

    return loader_train, loader_val

In [8]:
#@title train_gating_model
from gatednet import gnet, gnet_s, gnet_l, gnet_v
import torch.optim as optim
import torch.nn as nn
import shutil
from early_stopping import early_stop
import densenet as dn


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res
        
def train_gating_model(args, writer):    
    
    log_file = open(args.gated_models_dir+'/train_gated_nn_log.txt', 'a', newline='')
    log_file.write("Training gated nn mode:\n")
    log_file.write("loss threshold: " + str(args.loss_threshold) + "\n")
    
    %matplotlib inline

    loader_train, loader_test = load_generated_data(args, smote=True, return_to_org_shape=True)
    #model = gnet((3, 32, 32), args.num_ee + 1, args.filters)
    #model = gnet_s((3, 32, 32), args.num_ee + 1, args.filters)
    model = gnet_v((3, 32, 32), args.num_ee + 1)
    # create model
    #model = dn.DenseNet3(15,#args.layers, 
    #                     11, 
    #                     12,#args.growth, 
    #                     reduction=1,#args.reduce,
    #                     bottleneck=0,#args.bottleneck, 
    #                     dropRate=0)#args.droprate)
    #model = gnet_l(3*32*32, args.num_ee + 1)
    #ipdb.set_trace(context=6) 
    early_stopping = early_stop(patience=7)
    
    #args.device = 'cpu'
    model = model.to(args.device)
    if args.optimizer == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum,
                              weight_decay=args.weight_decay,
                              nesterov=True)
    else:
        optimizer = optim.Adam(model.parameters(), lr=args.lr)
    #criterion = nn.CrossEntropyLoss()
        
    print('Running for {:5d} epochs'.format(args.epochs))
    try:    
        torch.cuda.empty_cache()
        #ipdb.set_trace(context=6) 
                    
        for epoch in range(1, args.epochs + 1):
            print('{:3d}:'.format(epoch), end='')
            losses = []
            model.train()
            for batch_id, (data, target) in enumerate(loader_train):
                data, target = data.to(args.device), target.to(args.device, dtype=torch.int64)
                
                #ipdb.set_trace(context=6) 
                pred = model(data)
                #writer.add_graph(model, data)
    
                #loss = criterion(pred, target)
                loss = F.nll_loss(pred.log(), target)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                losses.append(float(loss))
                  
            train_epoch_loss = round(np.mean(losses), 4)
            print('Train avg loss: {:.4f}'.format(train_epoch_loss))
            writer.add_scalar("Train/Loss", train_epoch_loss, epoch)
            for name, weight in model.named_parameters():
                writer.add_histogram(name,weight, epoch)
                #writer.add_histogram(f'{name}.grad',weight.grad, epoch)
                
            if epoch % 1 == 0:
                accs = []
                val_losses = []
                records = []
            
                model.eval()
                starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
                with torch.no_grad():
                    for batch_id, (data, target) in enumerate(loader_test):
                        data, target = data.to(args.device), target.to(args.device, dtype=torch.int64)
                        #ipdb.set_trace(context=6) 
                        starter.record()
                        pred = model(data)
                        ender.record()
                        torch.cuda.synchronize()
                        record = starter.elapsed_time(ender)
                        #loss = criterion(pred, target)
                        loss = F.nll_loss(pred.log(), target)
                        prec1 = accuracy(pred.float().data, target)[0]
                        
                        accs.append(float(prec1))
                        val_losses.append(float(loss))
                        records.append(float(record/args.test_batch))
                        #ipdb.set_trace(context=6)                         
                        
                    #ipdb.set_trace(context=6) 
                    print('\tValidation avg loss: {:.4f}, Avg test acc is: {:.2f}%, time: {:.4f}ms'.format(round(np.mean(val_losses), 4),\
                                                                                                           round(np.mean(accs), 4),\
                                                                                                           round(np.mean(records), 4)))
                    val_epoch_loss = round(np.mean(val_losses), 4)
                    writer.add_scalar("Validation/Loss", val_epoch_loss, epoch)
                    writer.add_scalar("Validation/Accuracy", round(np.mean(accs), 4), epoch)                
                    writer.add_scalar("Validation/Time", round(np.mean(records), 4), epoch)                

                    early_stopping(val_epoch_loss)
                    if early_stopping.early_stop:
                        break
                
                    accs = None
                    val_losses = None
                    records = None

            writer.flush()
        writer.close()

    except KeyboardInterrupt:
        print('exception')
        writer.close()
        sys.exit()
    
    
    filename = 'gated_models/' + str(args.dataset) + '/model.' + str(args.loss_threshold) + '.pt'
    print('saving gated model: ', filename)
    torch.save(model, filename)

In [9]:
#@title main

"""Main function of the program.

The function loads the dataset and calls training and validation functions.
"""
%load_ext autoreload
%autoreload 2

import importlib
importlib.reload(config)
importlib.reload(loss_functions)
importlib.reload(utils)
from enum import Enum

class Mode(Enum):
    train_main = 0
    train_ee = 1
    generate_exits_gt = 2 
    train_gating = 3
    generate_relative_loss = 4 
    plot_relative_loss = 5
    calc_relative_time = 6

def main(mode: Mode, writer):
    print(mode.value)
    args = config.args_global
    args += config.argu[mode.value]
    %pwd
    model, optimizer, lr_scheduler, args = initializer(args)
    print(args)
    #ipdb.set_trace(context=6) 
    
    
    if mode == Mode.train_gating:
        print('Training Gated model')
        accus = []
        for thresh in range(2, 22, 3):
            args.loss_threshold = thresh/10
            val_accu = train_gating_model(args, writer)
            accus.append(val_accu)
        print('val_accuracies: ', np.array(accus))
        #train_gating_model(args, model, train_loader, test_loader, trainset)
    
    print('Finished')

In [10]:
tensorboard_dir = 'runs/gated_experiment_1'
if os.path.exists(tensorboard_dir):
    shutil.rmtree(tensorboard_dir)
writer = SummaryWriter(tensorboard_dir)
%load_ext tensorboard
#%tensorboard --logdir runs/gated_experiment_1

mode = Mode.train_gating
main(mode, writer)

3
use cuda:  True  device:  cuda
full_flow False
last model found and loaded:  runs/models/cifar10/eenet110/ee10_fine_conv2_lambda_1.0/model.v4.pt
ee-block-0: flops=19.78 MMac, params=19.34 k, cost-rate=0.08
ee-block-1: flops=34.23 MMac, params=33.35 k, cost-rate=0.13
ee-block-2: flops=43.86 MMac, params=42.7 k, cost-rate=0.17
ee-block-3: flops=53.49 MMac, params=52.04 k, cost-rate=0.21
ee-block-4: flops=63.13 MMac, params=61.39 k, cost-rate=0.25
ee-block-5: flops=72.76 MMac, params=70.73 k, cost-rate=0.28
ee-block-6: flops=82.4 MMac, params=80.08 k, cost-rate=0.32
ee-block-7: flops=92.03 MMac, params=89.42 k, cost-rate=0.36
ee-block-8: flops=100.48 MMac, params=136.57 k, cost-rate=0.39
ee-block-9: flops=110.01 MMac, params=173.69 k, cost-rate=0.43
exit-block: flops=256.32 MMac, params=1.73 M, cost-rate=1.00
Namespace(adaptive_lr=False, add_noise=False, batch_size=128, clear_dirs=False, dataset='cifar10', device=device(type='cuda'), distribution='fine', ee_costs=None, epochs=300, exit_

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  
